In [2]:
import pandas as pd
import numpy as np

In [3]:
annual_crime = pd.read_pickle('data/pickled/annual_crime.pkl')
monthly_crime = pd.read_pickle('data/pickled/monthly_crime.pkl')

In [4]:
annual_crime.head()

,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,19373,17987,17758,16636,16049,18190,18881,19393,19131
1,15999,14661,15317,14037,13661,15981,17284,17057,17024
2,18107,16922,16563,16177,15576,17963,18224,19336,18915
3,17748,16420,16276,15699,15404,16904,18570,18683,19026
4,17699,16923,17468,16573,16695,17890,19002,19366,19822


In [5]:
monthly_crime.head()

,crime_count
Date,
2010-01-01,19373
2010-02-01,15999
2010-03-01,18107
2010-04-01,17748
2010-05-01,17699


In [ ]:
# X = crime_volume['crime_count']
# X = X[np.logical_not(np.isnan(X))]
# split = round(len(X) / 2)
# X1, X2 = X[0:split], X[split:]
# mean1, mean2 = X1.mean(), X2.mean()
# var1, var2 = X1.var(), X2.var()
# print('mean1=%f, mean2=%f' % (mean1, mean2))
# print('variance1=%f, variance2=%f' % (var1, var2))